# task definition
Train a model to classify a movie rewiew in positive or negative polarity.

# data collection
dataset of IMDb, available at: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

it uses 'wget' python module. Use ```pip install wget``` to install it!


In [1]:
import wget
import os
import tarfile
import numpy

numpy.random.seed(42)

filename = "dataset/aclImdb_v1.tar.gz"

dataset_link = "http://ai.stanford.edu/~amaas/data/sentiment/{}".format("aclImdb_v1.tar.gz")
try:
    os.mkdir("dataset")
except OSError:
    pass

if not os.path.isfile(filename):
    file = wget.download(dataset_link, out='dataset/aclImdb_v1.tar.gz')

    tar = tarfile.open(filename, "r:gz")
    tar.extractall("dataset")
    tar.close()

# data load and visualization

In [2]:
# montar dataset com valores de 1 a 4 e de 7 a 10, ao invés de binário
dataset_path = 'dataset/aclImdb'
train_positive_files = ['train/pos/'+f for f in os.listdir(dataset_path+'/train/pos') \
                        if os.path.isfile(os.path.join(dataset_path+'/train/pos', f))]

train_negative_files = ['train/neg/'+f for f in os.listdir(dataset_path+'/train/neg') \
                        if os.path.isfile(os.path.join(dataset_path+'/train/neg', f))]

test_positive_files = ['test/pos/'+f for f in os.listdir(dataset_path+'/test/pos') \
                       if os.path.isfile(os.path.join(dataset_path+'/test/pos', f))]

test_negative_files = ['test/neg/'+f for f in os.listdir(dataset_path+'/test/neg') \
                       if os.path.isfile(os.path.join(dataset_path+'/test/neg', f))]

all_files = list(set().union(train_positive_files,train_negative_files, test_positive_files, test_negative_files))

In [3]:
dataset = {'trainset':[], 
           'polarity':[], 
           'bin_polarity': [],
           'review':[]}

for file in all_files:
    polarity = file.split('.')[0].split('_')[1]
    with open(os.path.join(dataset_path, file), 'r') as text_file:
        dataset['trainset'].append(file.split('/')[0])
        bin_polarity = 1 if int(polarity) > 5 else 0  # transform into binary polarity
        dataset['bin_polarity'].append(bin_polarity)
        dataset['polarity'].append(polarity)
        dataset['review'].append(text_file.readlines()[0])

In [4]:
import pandas as pd
dataframe = pd.DataFrame(data=dataset)

In [5]:
dataframe.head()

,trainset,polarity,bin_polarity,review
0,train,8,1,Too many sources routinely lump this thought-p...
1,test,3,0,"Yesterday I watched this tv production, and I ..."
2,train,10,1,"So wonderful, so quirky, so romantic, so Itali..."
3,train,2,0,"hg is normally exploitive, and it's never real..."
4,test,1,0,Dire beyond belief. Obviously set on the Isle ...


In [6]:
import matplotlib.pyplot as plt

dataframe['trainset'].value_counts().plot(kind='bar',title='Trainsets')
plt.show()

dataframe['polarity'].value_counts().plot(kind='bar',title='Polarities')
plt.show()

dataframe['bin_polarity'].value_counts().plot(kind='bar',title='Polarities binary')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

# pre processing

In [7]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(review):
    review = REPLACE_NO_SPACE.sub("", review.lower())
    review = REPLACE_WITH_SPACE.sub(" ", review)
    
    return review

dataframe['clean_review'] = dataframe['review'].apply(preprocess_reviews)

In [8]:
dataframe.head()

,trainset,polarity,bin_polarity,review,clean_review
0,train,8,1,Too many sources routinely lump this thought-p...,too many sources routinely lump this thought p...
1,test,3,0,"Yesterday I watched this tv production, and I ...",yesterday i watched this tv production and i w...
2,train,10,1,"So wonderful, so quirky, so romantic, so Itali...",so wonderful so quirky so romantic so italian ...
3,train,2,0,"hg is normally exploitive, and it's never real...",hg is normally exploitive and its never really...
4,test,1,0,Dire beyond belief. Obviously set on the Isle ...,dire beyond belief obviously set on the isle o...


# feature engineering

In [9]:
train_reviews = dataframe[dataframe['trainset'] == 'train']['clean_review'].values.tolist()
train_classes = dataframe[dataframe['trainset'] == 'train']['bin_polarity'].values.tolist()
test_reviews = dataframe[dataframe['trainset'] == 'test']['clean_review'].values.tolist()
test_classes = dataframe[dataframe['trainset'] == 'test']['bin_polarity'].values.tolist()

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

cv = CountVectorizer(binary=True)
cv.fit(train_reviews)
X = cv.transform(train_reviews)
X_test = cv.transform(test_reviews)

In [11]:
#save cv model
try:
    os.mkdir("models")
except OSError:
    pass

s = pickle.dumps(cv)
with open('models/cv_pickle', 'wb') as pickle_file:
    pickle_file.write(s)

# Training

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, train_classes, train_size = 0.75
)

/Users/erickmaziero/virtualenvs/PLN+IA_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
for c in [0.001, 0.005, 0.009, 0.01, 0.015, 0.02, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    

Accuracy for C=0.001: 0.85008
Accuracy for C=0.005: 0.8728
Accuracy for C=0.009: 0.8808
Accuracy for C=0.01: 0.88176
Accuracy for C=0.015: 0.8848
Accuracy for C=0.02: 0.88784
Accuracy for C=1: 0.8816


In [14]:
final_model = LogisticRegression(C=0.02)
final_model.fit(X, train_classes)
print ("Final Accuracy: %s" 
       % accuracy_score(test_classes, final_model.predict(X_test)))

Final Accuracy: 0.88116


In [15]:
s = pickle.dumps(final_model)
with open('models/final_model', 'wb') as pickle_file:
    pickle_file.write(s)

# evaluation

In [16]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
        
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

('excellent', 0.7700178186552931)
('perfect', 0.6438064663604238)
('great', 0.6257814304423217)
('amazing', 0.5128869629639836)
('wonderful', 0.5041690195174703)
('worst', -1.1322474910730729)
('waste', -0.8909325568277234)
('awful', -0.8265016623130904)
('boring', -0.6949326131326239)
('bad', -0.6472189426851046)


# deploying
install flask using ```pip3 install flask```. Let's create a REST service to make it available!
The following code must be put in a python file ```app.py```. 

In the same directory of the previous file, run the commands bellow:

```chmod a+x app.py```

```python3 app.py```

In [17]:
from flask import Flask, request
from sklearn.feature_extraction.text import CountVectorizer
import re
import pickle

# regular expressions
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

# load vectorization model
pickle_file = open('models/cv_pickle', 'rb')
cv = pickle.loads(pickle_file.read())

# load ML model
pickle_file = open('models/final_model', 'rb')
final_model = pickle.loads(pickle_file.read())


# prepare input to be classified
def prepare_input(review):
    review = REPLACE_NO_SPACE.sub("", review.lower())
    review = REPLACE_WITH_SPACE.sub(" ", review)

    return cv.transform([review])


app = Flask(__name__)

@app.route('/api/v1.0/scorefy', methods=["GET"])
def scorefy():
    query_parameters = request.args
    review = query_parameters.get('r')
    return final_model.predict(prepare_input(review))[0]


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=3000, debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/Users/erickmaziero/virtualenvs/PLN+IA_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
